In [ ]:
# Set and create directories for the TAO Toolkit experiment
import os

os.environ['PROJECT_DIR']='/dli/task/tao_project'
os.environ['SOURCE_DATA_DIR']='/dli/task/data'
os.environ['DATA_DIR']='/dli/task/tao_project/data'
os.environ['MODELS_DIR']='/dli/task/tao_project/models'
os.environ['SPEC_FILES_DIR']='/dli/task/spec_files'

In [ ]:
# Read the nvinfer config file
!cat $SPEC_FILES_DIR/pgie_config_resnet18_detector_unpruned.txt

In [ ]:
# Run the app_04.py DeepStream pipeline w/ the custom ResNet18 model
!nvidia-smi dmon -i 0 \
                 -s ucmt \
                 -c 20 > '/dli/task/logs/smi.log' & \
python sample_apps/app_04.py /dli/task/data/sample_30.h264 \
                            /dli/task/spec_files/pgie_config_resnet18_detector_unpruned.txt \
                            8 \
                            output_tiled.mp4

In [ ]:
# Convert the output video to a format that is compatible with Jupyter Lab
!ffmpeg -i output_tiled.mp4 output_tiled_conv.mp4 -y -loglevel quiet

In [ ]:
# Show video
from IPython.display import Video
Video('output_tiled_conv.mp4', width=720)

In [ ]:
# Read the smi.log
!cat logs/smi.log

In [ ]:
# View prune usage
!detectnet_v2 prune --help

In [ ]:
# Create a new ResNet model folder and prune the resnet18_detector model
!rm -rf $MODELS_DIR/resnet18_detector_pruned
!mkdir -p $MODELS_DIR/resnet18_detector_pruned

!detectnet_v2 prune -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.hdf5 \
                    -o $MODELS_DIR/resnet18_detector_pruned/resnet18_detector_pruned.hdf5 \
                    -k tlt_encode

In [ ]:
# List the model and sizes
!ls -rlt $MODELS_DIR/resnet18_detector/weights

!ls -rlt $MODELS_DIR/resnet18_detector_pruned

In [ ]:
# Evaluate the unpruned model
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config.txt \
                       -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.hdf5 \
                       -k tlt_encode

In [ ]:
# Evaluate the pruned model
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config.txt \
                       -m $MODELS_DIR/resnet18_detector_pruned/resnet18_detector_pruned.hdf5 \
                       -k tlt_encode

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/model_config_qat.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/training_config_qat.txt

In [ ]:
# UPDATED enable_qat and regularizer from training_config
# UPDATED pretrained_model_file and load_graph from model_config
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/dataset_config.txt \
     $SPEC_FILES_DIR/augmentation_config.txt \
     $SPEC_FILES_DIR/model_config_qat.txt \
     $SPEC_FILES_DIR/bbox_rasterizer_config.txt \
     $SPEC_FILES_DIR/postprocessing_config.txt \
     $SPEC_FILES_DIR/training_config_qat.txt \
     $SPEC_FILES_DIR/cost_function_config.txt \
     $SPEC_FILES_DIR/evaluation_config.txt \
     > $SPEC_FILES_DIR/combined_training_config_qat.txt
!cat $SPEC_FILES_DIR/combined_training_config_qat.txt

In [ ]:
# Initiate the training process
!detectnet_v2 train -e $SPEC_FILES_DIR/combined_training_config_qat.txt \
                    -r $MODELS_DIR/resnet18_detector_pruned_retrained_qat \
                    -k tlt_encode \
                    -n resnet18_detector_pruned_retrained_qat

In [ ]:
# List the newly retrained model
!ls -rlt $MODELS_DIR/resnet18_detector_pruned_retrained_qat/weights

In [ ]:
# Evaluate the model using the same validation set as training
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config_qat.txt \
                       -m $MODELS_DIR/resnet18_detector_pruned_retrained_qat/weights/resnet18_detector_pruned_retrained_qat.hdf5 \
                       -k tlt_encode

In [ ]:
# Delete duplicate copies
!rm -rf $MODELS_DIR/resnet18_detector_final/resnet18_detector_pruned_retrained_qat.hdf5
!rm -rf $MODELS_DIR/resnet18_detector_final/cal.bin

# Export the QAT trained model
!detectnet_v2 export -m $MODELS_DIR/resnet18_detector_pruned_retrained_qat/weights/resnet18_detector_pruned_retrained_qat.hdf5 \
                     -e $SPEC_FILES_DIR/combined_training_config_qat.txt \
                     -o $MODELS_DIR/resnet18_detector_final/resnet18_detector_pruned_retrained_qat \
                     -k tlt_encode \
                     --cal_cache_file $MODELS_DIR/resnet18_detector_final/cal.bin \
                     --data_type int8 \
                     --gen_ds_config

In [ ]:
# Run the app_04.py DeepStream pipeline w/ the pruned ResNet18 model
!nvidia-smi dmon -i 0 \
                 -s ucmt \
                 -c 20 > '/dli/task/logs/smi.log' & \
python sample_apps/app_04.py /dli/task/data/sample_30.h264 \
                            spec_files/pgie_config_resnet18_detector_optimized.txt \
                            16 \
                            output_tiled_optimized.mp4

In [ ]:
# Read the smi.log
!cat logs/smi.log

In [ ]:
# Convert the output video to a format that is compatible with Jupyter Lab
!ffmpeg -i output_tiled_optimized.mp4 output_tiled_optimized_conv.mp4 -y -loglevel quiet

In [ ]:
# Show video
Video('output_tiled_optimized_conv.mp4', width=720)